In [1]:
import os
import pathlib
import numpy as np
from PIL import Image
import tensorflow as tf

In [2]:
"""
# Mount the remote storage with the dataset
from google.colab import drive
drive.mount('/content/drive')
os.system("unzip -q drive/MyDrive/msa/CatsDogs.zip")
"""

'\n# Mount the remote storage with the dataset\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\nos.system("unzip -q drive/MyDrive/msa/CatsDogs.zip")\n'

# The first architecture

In [2]:
# Define the directory of the dataset
data_dir = pathlib.Path('CatsDogs/')

# Remove corrupted files
os.system("rm CatsDogs/Cats/666.jpg CatsDogs/Dogs/11702.jpg CatsDogs/Dogs/11410.jpg")

# Collects the path of all the files within the dataset
data_paths = [str(path) for path in list(data_dir.glob("*/*.jpg"))]
print(f"Images in the dataset: {len(data_paths)}")

Images in the dataset: 24997


rm: CatsDogs/Cats/666.jpg: No such file or directory
rm: CatsDogs/Dogs/11702.jpg: No such file or directory
rm: CatsDogs/Dogs/11410.jpg: No such file or directory


In [3]:
# Convert non-jpeg images into jpeg files
formats = [(path, Image.open(path).format) for path in data_paths]
non_jpegs = list(filter(lambda x: x[1]!='JPEG', formats))
for path, _ in non_jpegs:
    img = Image.open(path)
    img.convert('RGB').save(path, format='JPEG')

/Users/filippouslenghi/miniconda3/envs/audio2/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:822: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [4]:
# Create the respective tf.data.Dataset object
dataset = tf.data.Dataset.from_tensor_slices(data_paths)
# Shuffle the dataset
dataset = dataset.shuffle(len(data_paths))

Metal device set to: Apple M1 Pro


2022-07-09 15:36:54.562454: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-09 15:36:54.562940: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Get the class names
class_names = np.array(sorted([item.name for item in data_dir.glob('*') if item.name[0] != '.']))
print(class_names)

['Cats' 'Dogs']


In [6]:
# Create a test set
test_size = int(len(list(dataset)) * 0.2)
train = dataset.skip(test_size)
test = dataset.take(test_size)

# Create a validation set
val_size = int(len(list(train)) * 0.2)
train = train.skip(val_size)
val = train.take(val_size)

In [7]:
# Set initial params for the loader
batch_size = 64
img_height = 150
img_width = 150

In [8]:
def get_label(file_path):
    # Convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    one_hot = parts[-2] == class_names
    # Integer encode the label
    return tf.argmax(one_hot)

In [9]:
def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    return tf.image.resize(img, [img_height, img_width])

In [10]:
def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [11]:
# Create a dataset of image, label pairs
train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
val = val.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

In [12]:
# Configure dataset for performance
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

train = configure_for_performance(train)
val = configure_for_performance(val)
test = configure_for_performance(test)

In [13]:
num_classes = 1

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='sigmoid')
])

In [14]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

In [16]:
n_epochs = 10

model.fit(
    train,
    validation_data=val,
    epochs=n_epochs
)

Epoch 1/10


2022-07-09 00:43:08.240377: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-09 00:43:08.240519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 68/250 [=======>......................] - ETA: 9s - loss: 0.6990 - accuracy: 0.5191

Corrupt JPEG data: 252 extraneous bytes before marker 0xd9


118/250 [=============>................] - ETA: 6s - loss: 0.6836 - accuracy: 0.5515

...

222/250 [=========================>....] - ETA: 1s - loss: 0.6379 - accuracy: 0.6130

Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


250/250 [==============================] - ETA: 0s - loss: 0.6357 - accuracy: 0.6163

2022-07-09 00:43:21.357242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9


250/250 [==============================] - 15s 56ms/step - loss: 0.6357 - accuracy: 0.6163 - val_loss: 0.5805 - val_accuracy: 0.6924
Epoch 2/10
250/250 [==============================] - 13s 53ms/step - loss: 0.5266 - accuracy: 0.7354 - val_loss: 0.4893 - val_accuracy: 0.7642
Epoch 3/10
250/250 [==============================] - 13s 52ms/step - loss: 0.4683 - accuracy: 0.7788 - val_loss: 0.4236 - val_accuracy: 0.8147
Epoch 4/10
250/250 [==============================] - 13s 53ms/step - loss: 0.4181 - accuracy: 0.8072 - val_loss: 0.3884 - val_accuracy: 0.8250
Epoch 5/10
250/250 [==============================] - 13s 53ms/step - loss: 0.3686 - accuracy: 0.8336 - val_loss: 0.3487 - val_accuracy: 0.8552
Epoch 6/10
250/250 [==============================] - 13s 53ms/step - loss: 0.3294 - accuracy: 0.8558 - val_loss: 0.3613 - val_accuracy: 0.8382
Epoch 7/10
250/250 [==============================] - 13s 53ms/step - loss: 0.2730 - accuracy: 0.8854 - val_loss: 0.3392 - val_accuracy: 0.8460
Epo

In [17]:
model.evaluate(test)

44/79 [===============>..............] - ETA: 0s - loss: 0.2601 - accuracy: 0.9094

Corrupt JPEG data: 396 extraneous bytes before marker 0xd9


79/79 [==============================] - 2s 24ms/step - loss: 0.2696 - accuracy: 0.9104


[0.269606351852417, 0.910382091999054]

In [18]:
def zero_one_loss(dataset):

    missmatches = 0
    for i, (data, labels) in enumerate(dataset.as_numpy_iterator()):

        y_pred_binary = np.around(model.predict(x=np.asarray(data), verbose=0)).flatten()
        missmatches += np.sum(np.logical_xor(y_pred_binary, labels))


    return missmatches

In [19]:
print(f"Zero-one loss on the test set: {zero_one_loss(test)}")

2022-07-09 00:31:16.622534: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Zero-one loss on the test set: 535


2022-07-09 00:31:19.852043: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


# K-fold cross validation

In [21]:
from random import shuffle
from sklearn.model_selection import KFold



shuffle(data_paths)
k_fold = KFold(n_splits=5)
k_splits = k_fold.split(data_paths)
results = []
zero_one_losses = []

for train_index, test_index in k_splits:

    # Clear any previous state
    del model
    tf.keras.backend.clear_session()

    # Get the paths to the data
    train_paths = np.asarray(data_paths)[train_index]
    test_paths = np.asarray(data_paths)[test_index]

    # Make it tf.data.Dataset
    train = tf.data.Dataset.from_tensor_slices(train_paths)
    test = tf.data.Dataset.from_tensor_slices(test_paths)
    
    # Get labels
    train = train.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    test = test.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Configure for performance
    train = configure_for_performance(train)
    test = configure_for_performance(test)

    # Create the model
    num_classes = 1
    model = tf.keras.Sequential([
        tf.keras.layers.Rescaling(1./255),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])

    model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])
    
    model.fit(
        train,
        epochs=n_epochs,
        verbose=0
    )
    
    results.append(model.evaluate(test))
    zero_one_losses.append(zero_one_loss(test))

2022-07-08 20:24:17.585457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9
2022-07-08 20:26:53.667301: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


53/79 [===================>..........] - ETA: 0s - loss: 0.7847 - accuracy: 0.7948

Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9


79/79 [==============================] - 2s 25ms/step - loss: 0.7546 - accuracy: 0.8008


2022-07-08 20:26:56.123640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-08 20:26:59.514273: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-08 20:27:01.203628: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9
20

...

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9


20/79 [======>.......................] - ETA: 1s - loss: 0.5955 - accuracy: 0.8094

Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


79/79 [==============================] - 3s 29ms/step - loss: 0.5123 - accuracy: 0.8172


2022-07-08 20:37:52.788940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-07-08 20:37:56.850083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [24]:
mean_loss = np.mean(zero_one_losses)
mean_accuracy = np.mean([result[1] for result in results])
print(f'Mean zero-one loss: {mean_loss}, mean accuracy: {np.round(mean_accuracy, decimals=4)}')

Mean zero-one loss: 954.2, mean accuracy: 0.8091
